In [49]:
from pulp import *
from itertools import chain, combinations
import time
%run pravljenjeGrafova.ipynb


# funkcija za podskupove
def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))


# funkcija za susjede
def neighbors(x):
    n = []
    for (i,j) in Eweak:
        if i==x and Eweak[i,j] == 1:
            n.append(j)
    return n


# susjedstvo
def create_adjacency_dict(graph):
    adjacency_dict = {}
    
    for vertex, neighbors in graph.items():
        for neighbor in neighbors:
            if int(vertex)!=int(neighbor):
                adjacency_dict[(int(vertex), int(neighbor))] = 1
    
    for vertex in graph:
        for other_vertex in graph:
            if (int(vertex), int(other_vertex)) not in adjacency_dict and int(vertex)!=int(other_vertex) :
                adjacency_dict[(int(vertex), int(other_vertex))] = 0
    
    return adjacency_dict
            
    

In [50]:
g = napraviGraf("veoma mali grafovi/graf-14.txt")

dictE= create_adjacency_dict(g)
Eweak = dict(sorted(dictE.items()))

V=[*range(1,len(g)+1,1)]

print(Eweak)


{(1, 2): 1, (1, 3): 1, (1, 4): 0, (1, 5): 0, (1, 6): 0, (1, 7): 0, (1, 8): 0, (1, 9): 0, (1, 10): 0, (1, 11): 0, (1, 12): 0, (1, 13): 1, (1, 14): 1, (2, 1): 1, (2, 3): 1, (2, 4): 1, (2, 5): 1, (2, 6): 1, (2, 7): 1, (2, 8): 1, (2, 9): 0, (2, 10): 0, (2, 11): 0, (2, 12): 0, (2, 13): 0, (2, 14): 0, (3, 1): 1, (3, 2): 1, (3, 4): 0, (3, 5): 0, (3, 6): 1, (3, 7): 1, (3, 8): 0, (3, 9): 0, (3, 10): 0, (3, 11): 0, (3, 12): 0, (3, 13): 0, (3, 14): 0, (4, 1): 0, (4, 2): 1, (4, 3): 0, (4, 5): 1, (4, 6): 0, (4, 7): 0, (4, 8): 0, (4, 9): 0, (4, 10): 0, (4, 11): 0, (4, 12): 0, (4, 13): 0, (4, 14): 0, (5, 1): 0, (5, 2): 1, (5, 3): 0, (5, 4): 1, (5, 6): 0, (5, 7): 0, (5, 8): 1, (5, 9): 0, (5, 10): 0, (5, 11): 0, (5, 12): 0, (5, 13): 0, (5, 14): 0, (6, 1): 0, (6, 2): 1, (6, 3): 1, (6, 4): 0, (6, 5): 0, (6, 7): 1, (6, 8): 0, (6, 9): 1, (6, 10): 0, (6, 11): 0, (6, 12): 0, (6, 13): 0, (6, 14): 0, (7, 1): 0, (7, 2): 1, (7, 3): 1, (7, 4): 0, (7, 5): 0, (7, 6): 1, (7, 8): 1, (7, 9): 0, (7, 10): 0, (7, 11): 0,

In [51]:

# model - problem minimizacije
startTime = time.time()
model = LpProblem("MinimumWeaklyConnectedDominatingSet", LpMinimize)

# varijable
x = LpVariable.dicts("x", Eweak, 0, 1, LpBinary)
y = LpVariable.dicts("y", V , 0, 1, LpBinary)

# funkcija cilja
model += 1 + lpSum( x [ (u, v) ]  for (u, v) in Eweak )

# ogranicenja 1, 2 i 3
for u in V:
    for v in V:
        if u!=v:
            model += x[ (u, v) ] <= Eweak[u,v]
            model += x[ (u, v) ] == x[ (v, u) ]
            model += y[u] + y[v] >= 2*x[ (u, v) ]

# ogranicenje 4
for S in powerset(V):
    for u in S:
        for v in S:
            if u!=v and len(S)>2:
                model += lpSum( x[ (u, v) ] ) <= len(S)-1

# ogranicenje 5
for u in V:
    for v in V:
        if u!=v:
            model += lpSum( x[ (u,v) ]) == lpSum( y[t] for t in V) - 1

# ogranicenje 6
for v in V:
    for t in neighbors(v): 
        model += lpSum( y[t] ) >= 1


# rjesavanje

#model.solve(PULP_CBC_CMD(msg=1, maxSeconds=time_limit_in_seconds))
model.solve(PULP_CBC_CMD(maxSeconds=600, msg=1, fracGap=0))


# ispis DS
selected = [u for u, var in y.items() if var.varValue == 1]
selectedE = [u for u, var in x.items() if var.varValue == 3]
print("Selected:",selected)
print("Selected edges ",selectedE )
print("Selected:", len(selected))
print("--- %s sekundi ---" % (time.time() - startTime))





Selected: [2, 3, 10, 13]
Selected edges  [(1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (2, 1), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12), (2, 13), (2, 14), (3, 1), (3, 2), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (3, 11), (3, 12), (3, 13), (3, 14), (4, 1), (4, 2), (4, 3), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (4, 11), (4, 12), (4, 13), (4, 14), (5, 1), (5, 2), (5, 3), (5, 4), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (5, 11), (5, 12), (5, 13), (5, 14), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 7), (6, 8), (6, 9), (6, 10), (6, 11), (6, 12), (6, 13), (6, 14), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 8), (7, 9), (7, 10), (7, 11), (7, 12), (7, 13), (7, 14), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7), (8, 9), (8, 10), (8, 11), (8, 12), (8, 13), (8, 14), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (9, 6), (9, 7), (9, 8), (9, 10), (9, 11), (9, 